In [188]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [189]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Sabilov-FR").getOrCreate()

In [190]:
spark

In [191]:
!hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2021-02-27 22:13 /labs/slaba04/gender_age_dataset.txt


In [192]:
# !hdfs dfs -head /labs/slaba04/gender_age_dataset.txt

In [193]:
from pyspark.sql import functions as f
from pyspark.sql.types import *

schema = StructType() \
        .add('visits', ArrayType(StructType([
                                StructField('url', StringType()),
                                StructField('timestamp', LongType())
])), True)

df = spark.read.csv('/labs/slaba04/gender_age_dataset.txt', sep='\t', header=True)
df.printSchema()
# df.groupBy('gender').count().show()
# df.orderBy('age').groupBy('age').count().show()
# df.orderBy('age').groupBy('gender', 'age').count().show()
df = df.withColumn('data', f.from_json(f.col("user_json"), schema)).select(f.col('uid'), f.col('data.*'))
df.select(f.col('visits').getField(1).alias('first_obj'), 
          f.concat(f.lit('{"uid":'), f.col('uid'), f.lit(',"gender":"M","age":"25-34"}')).alias('value'))\
          .show(10, False, True)

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------
 first_obj | [http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story, 1426666298001] 
 value     | {"uid":d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"gender":"M","age":"25-34"}                                                 
-RECORD 1----------------------------------------------------------------------------------------------------------------------------
 first_obj | [http://sweetrading.ru/?p=884, 1419717884437]                                                                           
 value     | {"uid":d502331d-621e-4721-ada2-5d30b2c3801f,"gender":"M","age":"25-34"}                                                 
-RECORD 2--------------------------------

- |gender|  age|count|
- +------+-----+-----+
- |     -|    -| 5000|
- |     F|18-24| 2886|
- |     M|18-24| 2012|
- |     M|25-34| 8666|
- |     F|25-34| 6791|
- |     F|35-44| 4271|
- |     M|35-44| 5089|
- |     F|45-54| 2597|
- |     M|45-54| 2147|
- |     M| >=55|  784|
- |     F| >=55|  895|

In [198]:
from pyspark.sql import functions as f
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_farhad.sabilov",
    "startingOffsets": "earliest"
}

schema = StructType([
    StructField('uid', StringType(), False),
    StructField('visits', StringType(), False)
])


kafka_sdf = spark.read.format("kafka").options(**read_kafka_params).option("failOnDataLoss", 'False').load()
# kafka_sdf.withColumn("data", f.from_json(f.col("value").cast("string"), schema)).select(f.col("data.*"))\
#          .show(5)


sink = kafka_sdf.withColumn('data', f.from_json(f.col("value").cast('string'), schema)).select(f.col('data.*'))\
         .withColumn('gender', f.array(f.lit("F"), f.lit("M")).getItem((f.rand()*2).cast('int')))\
         .select(f.concat(f.lit('{"uid":'), 
                          f.col('uid'), 
                          f.lit(',"gender":'),
                          f.col('gender'),
                          f.lit(',"age":"25-34"}')
                         ).alias('value'))
                     
sink.show(20, False, True)

-RECORD 0----------------------------------------------------------------------
 value | {"uid":bd7a30e1-a25d-4cbf-a03f-61748cbe540e,"gender":M,"age":"25-34"} 
-RECORD 1----------------------------------------------------------------------
 value | {"uid":bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,"gender":F,"age":"25-34"} 
-RECORD 2----------------------------------------------------------------------
 value | {"uid":bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,"gender":M,"age":"25-34"} 
-RECORD 3----------------------------------------------------------------------
 value | {"uid":bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,"gender":M,"age":"25-34"} 
-RECORD 4----------------------------------------------------------------------
 value | {"uid":bd7e54a2-0215-45cb-a869-9efebf250e38,"gender":F,"age":"25-34"} 
-RECORD 5----------------------------------------------------------------------
 value | {"uid":bd7e9797-4cdb-46e1-a540-f3ea010605ad,"gender":M,"age":"25-34"} 
-RECORD 6-------------------------------

In [221]:
#0.2512

from pyspark.sql import functions as f
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_farhad.sabilov",
    "startingOffsets": "latest"
}

write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "farhad.sabilov"
}



schema = StructType([
    StructField('uid', StringType(), False),
    StructField('visits', StringType(), False)
])


kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).option("failOnDataLoss", 'False').load()
sink = kafka_sdf.withColumn('data', f.from_json(f.col("value").cast('string'), schema))\
                .select(f.col('data.*'))\
                .select(f.concat(f.lit('{"uid":"'), f.col('uid'), f.lit('","gender":"M","age":"25-34"}')).alias('value'))\
                .writeStream.format("kafka")\
                .options(**write_kafka_params)\
                .option("checkpointLocation", "streaming/chk/chk_kafka")\
                .outputMode("append")\
                .start()

In [222]:
# # 0.2176

# from pyspark.sql import functions as f
# read_kafka_params = {
#     "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
#     "subscribe": "input_farhad.sabilov",
#     "startingOffsets": "latest"
# }

# write_kafka_params = {
#    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
#    "topic": "farhad.sabilov"
# }



# schema = StructType([
#     StructField('uid', StringType(), False),
#     StructField('visits', StringType(), False)
# ])


# kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).option("failOnDataLoss", 'False').load()
# sink = kafka_sdf.withColumn('data', f.from_json(f.col("value").cast('string'), schema))\
#                 .select(f.col('data.*'))\
#                 .withColumn('gender', f.array(f.lit("F"), f.lit("M")).getItem((f.rand()*2).cast('int')))\
#                 .select(f.concat(f.lit('{"uid":"'), 
#                                           f.col('uid'), 
#                                           f.lit('","gender":"'),
#                                           f.col('gender'),
#                                           f.lit('","age":"25-34"}')
#                                          ).alias('value'))\
#                 .writeStream.format("kafka")\
#                 .options(**write_kafka_params)\
#                 .option("checkpointLocation", "streaming/chk/chk_kafka")\
#                 .outputMode("append")\
#                 .start()

In [223]:
sink.isActive

True

In [224]:
sink.status

{'message': 'Getting offsets from KafkaV2[Subscribe[input_farhad.sabilov]]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [225]:
sink.lastProgress

In [226]:
!/usr/hdp/current/kafka-broker/bin/kafka-console-consumer.sh --bootstrap-server spark-master-1.newprolab.com:6667 --topic farhad.sabilov

{"uid":"bd7a30e1-a25d-4cbf-a03f-61748cbe540e","gender":"M","age":"25-34"}
{"uid":"bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd","gender":"M","age":"25-34"}
{"uid":"bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c","gender":"M","age":"25-34"}
{"uid":"bd7c5d7a-0def-41d1-895f-fdb96c56c2d4","gender":"M","age":"25-34"}
{"uid":"bd7e54a2-0215-45cb-a869-9efebf250e38","gender":"M","age":"25-34"}
{"uid":"bd7e9797-4cdb-46e1-a540-f3ea010605ad","gender":"M","age":"25-34"}
{"uid":"bd7e9ec7-fb67-45eb-8ad3-209d01d15ae6","gender":"M","age":"25-34"}
{"uid":"bd8056df-cc25-4b63-bc12-a46f888baa49","gender":"M","age":"25-34"}
{"uid":"bd818690-73d2-445d-be5d-5c8f748dbb19","gender":"M","age":"25-34"}
{"uid":"bd81e006-f059-4cdd-b716-3467c78d1312","gender":"M","age":"25-34"}
{"uid":"bd81e64a-bfa3-4147-b094-e3d248c44e9a","gender":"M","age":"25-34"}
{"uid":"bd82fee4-afb3-4085-a5da-9bce2b5523ef","gender":"M","age":"25-34"}
{"uid":"bd83400b-abe2-42fd-b73a-4edb886fb036","gender":"M","age":"25-34"}
{"uid":"bd843c8c-dbba-4ecb-b5e8-45ba8c

{"uid":"470f4432-6f51-4065-9d14-faf855e43ac5","gender":"M","age":"25-34"}
{"uid":"470fad91-2cdf-4c45-9735-87d1584493c1","gender":"M","age":"25-34"}
{"uid":"4712ec83-9b4e-4c0d-8cd2-91d6e9ffac96","gender":"M","age":"25-34"}
{"uid":"471330f5-56e1-482d-bb86-3bfb68bf3a11","gender":"M","age":"25-34"}
{"uid":"47154764-ff86-4570-8e86-e13f7c01f54c","gender":"M","age":"25-34"}
{"uid":"47156cb9-ff6b-46b5-8219-a745889d071e","gender":"M","age":"25-34"}
{"uid":"4715d7b1-a5b0-4252-81a1-292eccb65a21","gender":"M","age":"25-34"}
{"uid":"4715d98c-fd4f-4b5d-a670-7faffbf741f3","gender":"M","age":"25-34"}
{"uid":"4715e945-9225-444d-a836-c39df2baebbd","gender":"M","age":"25-34"}
{"uid":"4716e702-c18d-4aab-9776-c05bb7585058","gender":"M","age":"25-34"}
{"uid":"4718970b-bdb1-47a5-9f5a-2b71e6cecdb2","gender":"M","age":"25-34"}
{"uid":"47194d6b-dcb3-4e5b-84b4-ca3fc827087c","gender":"M","age":"25-34"}
{"uid":"471ac4dd-84fd-4c70-9a6d-a6511ad20a18","gender":"M","age":"25-34"}
{"uid":"47228de2-0617-4b63-bd0e-5a600b

{"uid":"47e5d3da-4627-40a4-bdfc-7ef0b35370d0","gender":"M","age":"25-34"}
{"uid":"47e5d9fc-b7fd-4fd0-8320-d34355a009f7","gender":"M","age":"25-34"}
{"uid":"47e6127f-5f40-4fb9-be7b-9a2f41b28f02","gender":"M","age":"25-34"}
{"uid":"47e80c6b-d9a5-42fe-889e-79fcd40fe968","gender":"M","age":"25-34"}
{"uid":"47eb6f38-f3ca-49f5-bd77-5dd86d8b4bb2","gender":"M","age":"25-34"}
{"uid":"47eec013-0503-49c7-8c12-ff807189d8d7","gender":"M","age":"25-34"}
{"uid":"47eeff6d-3862-4451-965d-4c8520c9bdc9","gender":"M","age":"25-34"}
{"uid":"47f0716d-7ca4-43e6-8026-82e0d6b678f7","gender":"M","age":"25-34"}
{"uid":"47f0cd70-4bfe-4d90-b5fd-c209839ff6ac","gender":"M","age":"25-34"}
{"uid":"47f0de31-34f0-406e-a8de-14469e08be15","gender":"M","age":"25-34"}
{"uid":"47f29ff5-7dea-48af-90e4-5bb527d101b7","gender":"M","age":"25-34"}
{"uid":"47f6ef8a-ca48-446a-bbe6-aa4698973b59","gender":"M","age":"25-34"}
{"uid":"47fb70e9-62e1-4258-b1fa-125d3ed8cc7d","gender":"M","age":"25-34"}
{"uid":"47fc5feb-3ea0-45cc-9791-a65dbe

{"uid":"1d8bb2cd-3dd8-464e-a5db-6ee522a3d67d","gender":"M","age":"25-34"}
{"uid":"1d8c1e73-d4ad-4a52-a7f6-bb0b0d46efbc","gender":"M","age":"25-34"}
{"uid":"1d8cd0ee-7828-46f3-bb19-de94dcae68b8","gender":"M","age":"25-34"}
{"uid":"1d8cf189-7e96-4720-ae32-f0a9d97883f1","gender":"M","age":"25-34"}
{"uid":"1d8d6bfa-ab8f-4552-ac13-47e1f6d7a502","gender":"M","age":"25-34"}
{"uid":"1d9007c5-bc0b-43fd-9a8d-0896ac72cf23","gender":"M","age":"25-34"}
{"uid":"1d915cd2-ceb8-4b67-be4d-0b712c06bb0e","gender":"M","age":"25-34"}
{"uid":"1d941a3e-5059-4943-8a71-48f7793ea657","gender":"M","age":"25-34"}
{"uid":"1d973e09-e66a-4c8d-997f-496b250752a9","gender":"M","age":"25-34"}
{"uid":"1d9799ee-7193-4c88-9cd3-197554458ea2","gender":"M","age":"25-34"}
{"uid":"1d9ab658-25ef-478f-8156-5e87d0d9706b","gender":"M","age":"25-34"}
{"uid":"1d9e0730-e096-4fdc-8421-6b978f33aefb","gender":"M","age":"25-34"}
{"uid":"1d9e1351-8c2c-49be-9060-7fe4ade1b9bf","gender":"M","age":"25-34"}
{"uid":"1d9eab77-fed7-413f-bac6-44afa0

{"uid":"733744c9-cdf5-4f4d-b230-b5725fbb5419","gender":"M","age":"25-34"}
{"uid":"733af978-3fd2-49eb-b48e-9bde19a4625a","gender":"M","age":"25-34"}
{"uid":"733b2f85-c348-4add-a168-9e50078d8f31","gender":"M","age":"25-34"}
{"uid":"733b3c94-ec5a-46b2-8d99-a6c3216df9c9","gender":"M","age":"25-34"}
{"uid":"733c8cf7-1278-4a09-b746-b84dcaf85850","gender":"M","age":"25-34"}
{"uid":"733d1c59-2b7e-4edf-ab19-048c5a78eaec","gender":"M","age":"25-34"}
{"uid":"733e215e-f6dd-4b2f-acd9-1e401f749e65","gender":"M","age":"25-34"}
{"uid":"733ecaa4-b964-4f40-8cd3-6d9e1d2b858a","gender":"M","age":"25-34"}
{"uid":"733f0741-9098-43a7-b1a3-ba3a429ab073","gender":"M","age":"25-34"}
{"uid":"734244ad-a7a7-4df4-8ace-2bf3016a1545","gender":"M","age":"25-34"}
{"uid":"73424cd2-a77f-4f06-bc23-df42937dd055","gender":"M","age":"25-34"}
{"uid":"7343648a-ceb7-4491-a9db-88152b208b99","gender":"M","age":"25-34"}
{"uid":"73446f56-9cdd-4cea-8577-791a387dc82d","gender":"M","age":"25-34"}
{"uid":"7344d2fc-c224-48fb-b878-d53cf1

...

{"uid":"fd7428f0-32d0-4dae-be9d-a34109b7fc77","gender":"M","age":"25-34"}
{"uid":"fd74f717-1246-4834-b7d5-818677be57cd","gender":"M","age":"25-34"}
{"uid":"fd75a159-44fb-4098-8af2-6a90351054ef","gender":"M","age":"25-34"}
{"uid":"fd7913d6-7941-4ca9-b747-8098bd6eb59c","gender":"M","age":"25-34"}
{"uid":"fd794a1e-7c50-411a-a408-67b003de31e9","gender":"M","age":"25-34"}
{"uid":"fd7b63da-93d9-4c12-a09b-82d1e8797aa6","gender":"M","age":"25-34"}
{"uid":"fd7c1610-0e8c-482c-8645-edd75afc295a","gender":"M","age":"25-34"}
{"uid":"fd7f0b10-ff58-4610-8d10-7840e6dc21a5","gender":"M","age":"25-34"}
{"uid":"fd80cf8f-cf55-4c57-afd6-953c0a67380a","gender":"M","age":"25-34"}
{"uid":"fd82f354-352e-4b77-96ae-1eb9e13e5a58","gender":"M","age":"25-34"}
{"uid":"fd83bfef-b1bb-4015-9ebe-7c1c982da20e","gender":"M","age":"25-34"}
{"uid":"fd851df3-e948-4d3f-80bb-7c733a9109f8","gender":"M","age":"25-34"}
{"uid":"fd85fa27-2670-4b59-b299-e0e9b67b8fc7","gender":"M","age":"25-34"}
{"uid":"fd876afe-ecb7-411b-805a-0df749

{"uid":"fbc351a3-83bf-4d55-9d09-a96e4e04dcd4","gender":"M","age":"25-34"}
{"uid":"fbc434b1-1daf-46fc-9abd-7918150e89e3","gender":"M","age":"25-34"}
{"uid":"fbc4b142-ea31-4781-bb04-45a647b20602","gender":"M","age":"25-34"}
{"uid":"fbc5bc9d-ccf3-4df4-b118-f30eeec43320","gender":"M","age":"25-34"}
{"uid":"fbc61f58-a500-444c-b18a-fd5319d9d12f","gender":"M","age":"25-34"}
{"uid":"fbc85e48-3d51-48ad-b32c-466fd5947547","gender":"M","age":"25-34"}
{"uid":"fbccda84-9af5-445a-b8c2-e114e4b0c488","gender":"M","age":"25-34"}
{"uid":"fbce2d74-6a2a-4bf3-9b3f-ea98a85155ea","gender":"M","age":"25-34"}
{"uid":"fbce3d61-17ff-43ed-8566-8013df5f01ad","gender":"M","age":"25-34"}
{"uid":"fbce5409-ac4a-4be7-874a-7991c6b4a21d","gender":"M","age":"25-34"}
{"uid":"fbcffac2-0796-4a38-8200-16a5620cc834","gender":"M","age":"25-34"}
{"uid":"fbd065a5-7f32-4cc4-a571-0bc49e0775d7","gender":"M","age":"25-34"}
{"uid":"fbd130f9-9aa4-4c1e-b523-fc7405e06074","gender":"M","age":"25-34"}
{"uid":"fbd54753-dcf6-4390-a40b-64b489

{"uid":"133007de-b896-474d-a127-abb601dd21aa","gender":"M","age":"25-34"}
{"uid":"1332f255-a77b-48e8-b39e-082f35044485","gender":"M","age":"25-34"}
{"uid":"13365234-038f-4288-a936-2cc01a1eb57b","gender":"M","age":"25-34"}
{"uid":"13367262-a5ed-41b8-b2ac-832cc33ce8a8","gender":"M","age":"25-34"}
{"uid":"133e5929-383f-4d8e-bd94-5a63b6e13ab0","gender":"M","age":"25-34"}
{"uid":"133f2e43-b9b3-4d08-949f-f5a0fca03476","gender":"M","age":"25-34"}
{"uid":"133f35d8-0f2b-4df3-82eb-b4abcad7b110","gender":"M","age":"25-34"}
{"uid":"134043d9-a76f-4af9-9cbc-99cfc0c7d323","gender":"M","age":"25-34"}
{"uid":"1340fe81-1a5d-451d-8c62-a09897885f82","gender":"M","age":"25-34"}
{"uid":"13437ed2-e954-4557-bb96-fea9e445ec19","gender":"M","age":"25-34"}
{"uid":"1345571e-a201-49a3-8937-b2eeb8e46ff9","gender":"M","age":"25-34"}
{"uid":"13473941-1cd9-4c9f-ba3c-9d3f2305c788","gender":"M","age":"25-34"}
{"uid":"134ba92d-e774-47a0-ba6d-f9b5d3c201f4","gender":"M","age":"25-34"}
{"uid":"134d4299-d9a5-45a1-8ee4-837cc4

{"uid":"13f908e2-2ddc-4538-ac7a-00986b4b8ac8","gender":"M","age":"25-34"}
{"uid":"13f91463-8386-44cb-8740-a61652488b2e","gender":"M","age":"25-34"}
{"uid":"13f99fa3-a93a-4bb9-a67d-53143ad14837","gender":"M","age":"25-34"}
{"uid":"13fa2ca9-e5c1-45cd-9d42-a1d8a1057690","gender":"M","age":"25-34"}
{"uid":"13fa412d-b38e-4d8b-a54c-e81c78b995e7","gender":"M","age":"25-34"}
{"uid":"13fa87f0-54ce-48ec-a7a2-b0c547ca3558","gender":"M","age":"25-34"}
{"uid":"13fae001-42ea-4c05-b3fa-6844382e664f","gender":"M","age":"25-34"}
{"uid":"13fba782-c7eb-4be1-81a9-126e16aa05ba","gender":"M","age":"25-34"}
{"uid":"fe21844f-a8c5-476c-ae37-6b9fd2cb784d","gender":"M","age":"25-34"}
{"uid":"fe223361-57a5-4b20-8224-a817beab8919","gender":"M","age":"25-34"}
{"uid":"fe26ab28-29d1-4a4c-bfe2-4d9c94bd1bc4","gender":"M","age":"25-34"}
{"uid":"fe26c04d-a734-4ace-ad6e-e7a2b377dfc0","gender":"M","age":"25-34"}
{"uid":"fe26f24e-bec9-4348-b076-fab592965fff","gender":"M","age":"25-34"}
{"uid":"fe27653d-b6d9-4bb0-88b0-3ed17c

^C
Processed a total of 5000 messages


In [227]:
streams = SparkSession.builder.getOrCreate().streams.active
print(streams)
for s in streams:
    s.stop()

In [228]:
spark.stop()